In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
olympics = [("United States", "Gold", 39), 
             ("United States", "Silver", 41),
             ("United States", "Bronze", 33),
             ("China", "Gold", 38), 
             ("China", "Silver", 32),
             ("China", "Bronze", 19),
             ("Japan", "Gold", 27), 
             ("Japan", "Silver", 14),
             ("Japan", "Bronze", 17),
             ("Great Britain", "Gold", 22), 
             ("Great Britain", "Silver", 20),
             ("Great Britain", "Bronze", 22)          
             ]


col = ['Country', 'medal', 'count']

olympicsdf = spark.createDataFrame(data = olympics,schema = col)

olympicsdf.printSchema()
olympicsdf.show()

root
 |-- Country: string (nullable = true)
 |-- medal: string (nullable = true)
 |-- count: long (nullable = true)

+-------------+------+-----+
|      Country| medal|count|
+-------------+------+-----+
|United States|  Gold|   39|
|United States|Silver|   41|
|United States|Bronze|   33|
|        China|  Gold|   38|
|        China|Silver|   32|
|        China|Bronze|   19|
|        Japan|  Gold|   27|
|        Japan|Silver|   14|
|        Japan|Bronze|   17|
|Great Britain|  Gold|   22|
|Great Britain|Silver|   20|
|Great Britain|Bronze|   22|
+-------------+------+-----+



In [0]:
pivotDF = olympicsdf.groupBy("Country").sum("count")
pivotDF.printSchema()
pivotDF.show(truncate=False)

root
 |-- Country: string (nullable = true)
 |-- sum(count): long (nullable = true)

+-------------+----------+
|Country      |sum(count)|
+-------------+----------+
|United States|113       |
|China        |89        |
|Japan        |58        |
|Great Britain|64        |
+-------------+----------+



In [0]:
pivotDF = olympicsdf.groupBy("Country").pivot("medal").sum("count")
pivotDF.printSchema()
pivotDF.show(truncate=False)


pivotDF1 = olympicsdf.groupBy("Country").pivot("medal",['Gold','Silver']).sum("count")
pivotDF1.show(truncate=False)


root
 |-- Country: string (nullable = true)
 |-- Bronze: long (nullable = true)
 |-- Gold: long (nullable = true)
 |-- Silver: long (nullable = true)

+-------------+------+----+------+
|Country      |Bronze|Gold|Silver|
+-------------+------+----+------+
|Great Britain|22    |22  |20    |
|United States|33    |39  |41    |
|China        |19    |38  |32    |
|Japan        |17    |27  |14    |
+-------------+------+----+------+

+-------------+----+------+
|Country      |Gold|Silver|
+-------------+----+------+
|Great Britain|22  |20    |
|United States|39  |41    |
|China        |38  |32    |
|Japan        |27  |14    |
+-------------+----+------+



In [0]:
from pyspark.sql.functions import expr
unpivotExpr = "stack(3, 'G', Gold, 'S', Silver, 'B', Bronze) as (medal,Total)"
unPivotDF = pivotDF.select("Country", expr(unpivotExpr)) 
unPivotDF.show()

+-------------+-----+-----+
|      Country|medal|Total|
+-------------+-----+-----+
|Great Britain|    G|   22|
|Great Britain|    S|   20|
|Great Britain|    B|   22|
|United States|    G|   39|
|United States|    S|   41|
|United States|    B|   33|
|        China|    G|   38|
|        China|    S|   32|
|        China|    B|   19|
|        Japan|    G|   27|
|        Japan|    S|   14|
|        Japan|    B|   17|
+-------------+-----+-----+

